# Оглавление

# Часть 0 : Играем в песочнице

Установим все нужные библиотеки

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random as rnd
from tensorflow import keras
from tensorflow.keras import layers, models
import scipy as scp
import matplotlib.pyplot as plt
import zipfile
import json
%matplotlib inline

eps = 1e-10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Пока мы ещё маленькие и совсем ничего не понимаем в нейросетях. Сделаем простую игру (что-то в духе камень-ножницы-бумага), чтобы проверить, работает ли модель в принципе и научится ей пользоваться для более сложной задачи, которая ждёт нас впереди.

In [ ]:
def game_1():
    ### ЭТАП 1: МОДЕЛЬ ###
    model = models.Sequential([
        layers.InputLayer(shape=(2,)),  # Вход: 2 героя
        layers.Dense(16, activation='relu'),  # Скрытый слой
        layers.Dense(1, activation='sigmoid')  # Выход: Победитель (0 или 1)
    ])

    ### ЭТАП 2: ОБУЧЕНИЕ ###
    '''
    Генерируем матчи:
    a и b - от 1 до 3.
    Если они совпадают, то мы их не добавляем (мы пока не рассматриваем ничьи)

    '''
    X = np.array([
        [a, b] for _ in range(1500)
        for a, b in [(rnd.randint(1, 3), rnd.randint(1, 3))]
        if a != b
    ])
    Y = np.array([0 if a < b else 1 for a, b in X]) # Номер победившего в паре игрока

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X, Y, epochs=50, batch_size=10, verbose=0)

    ### ЭТАП 3: ТЕСТИРОВАНИЕ ###

    # Теперь создадим тестовые игры, чтобы проверить, что нейросеть обучилась
    X_test = np.array([
        [a, b] for _ in range(10)
        for a, b in [(rnd.randint(1, 3), rnd.randint(1, 3))]
        if a != b
    ])

    Y_test = np.array([0 if a < b else 1 for a, b in X_test]) # Правильные ответы
    y_pred = (model.predict(X_test) > 0.5).astype(int)  # Предсказания нашей модели
    errors = 0

    # Сравниваем
    for i in range(len(y_pred)):
        p1 = ['A', 'B', 'C'][X_test[i][0] - 1]
        p2 = ['A', 'B', 'C'][X_test[i][1] - 1]
        if y_pred[i] != Y_test[i]:
            print(f"Ошибка! {i+1}) {p1} против {p2}, победил не {p2 if y_pred[i] else p1}")
            errors += 1
        else:
            print(f"{i+1}) {p1} против {p2} => Победитель: {p2 if y_pred[i] else p1}")
    print("Процент ошибок:", round((errors / len(Y_test)) * 100, 2), "%")

In [ ]:
game_1()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1) B против A => Победитель: A
2) B против A => Победитель: A
3) A против C => Победитель: A
4) B против C => Победитель: B
5) C против A => Победитель: A
6) A против B => Победитель: A
7) B против A => Победитель: A
Процент ошибок: 0.0 %


P.s. В дальнейшем мы не будем так подробно расписывать комментарии

 Всё работает, отлично - модель научилась угадывать победителя. Теперь напишем, чтобы она сама участвовала в игре и выбирала себе героя.

In [ ]:
def game_2():
    np.random.seed(42)
    tf.random.set_seed(42)

    ### ЭТАП 1: МОДЕЛЬ ###
    def reward(a, b):
        return ([a, b] in [[0, 1], [0, 2], [1, 2]]) # 0 всегда выигрывает

    model = keras.Sequential([
        keras.layers.Input(shape=(1,)), # Вход : заглушка
        keras.layers.Dense(3, activation='softmax')  # Выход: 3 героя, p их выбора
    ])
    optimizer = keras.optimizers.Adam(learning_rate=0.01)

    ### ЭТАП 2: ОБУЧЕНИЕ ###
    num_episodes = 300   # Эпохи обучения
    games_per_episode = 50    # Матчей за одну эпоху

    for episode in range(num_episodes):
        with tf.GradientTape() as tape:
            # Вероятности для хода
            state_input = tf.ones((games_per_episode, 1), dtype=tf.float32)
            probs = model(state_input, training=True)
            log_probs = tf.math.log(probs + eps)

            # Генерируем ходы модели
            sampled_actions = tf.random.categorical(log_probs, num_samples=1)
            sampled_actions = tf.squeeze(sampled_actions, axis=1)

            # Считаем награду
            rewards = []
            for i in range(games_per_episode):
                model_hero = sampled_actions[i].numpy()
                possible_opp_moves = [x for x in [0,1,2] if x != model_hero]
                opp_hero = np.random.choice(possible_opp_moves)
                rewards.append(reward(model_hero, opp_hero))

            rewards = tf.constant(rewards, dtype=tf.float32)
            chosen_action_log_p = tf.reduce_sum(
                tf.one_hot(sampled_actions, depth=3) * log_probs,
                axis=1
            )
            loss = -tf.reduce_mean(rewards * chosen_action_log_p)

        # Корректируем модель, обновляем веса
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Выводим статистику
        if (episode + 1) % 20 == 0:
            mean_reward = tf.reduce_mean(rewards)
            print(f"Episode {episode + 1}/{num_episodes} | Mean reward: {round(mean_reward.numpy(), 2)} | Loss: {round(loss.numpy(), 3)}")

    ### ЭТАП 3: ТЕСТИРОВАНИЕ ###
    print('-' * 52 + '\nТестирование\n' + '-' * 52)
    N_test = 1000
    test_input = tf.ones((N_test, 1), dtype=tf.float32)
    test_probs = model(test_input, training=False)
    test_log_probs = tf.math.log(test_probs + 1e-10)
    test_actions = tf.random.categorical(test_log_probs, num_samples=1)
    test_actions = tf.squeeze(test_actions, axis=1).numpy()

    wins = 0
    L = ['A', 'B', 'C']
    for i in range(N_test):
        model_hero = test_actions[i]
        opp_hero = np.random.choice([x for x in [0, 1, 2] if x != model_hero])
        if reward(model_hero, opp_hero):
            wins += 1
            if i % 50 == 0:
                print(f"Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}")
        else:
            if i % 50 == 0:
                print(f"x - Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}")

    win_percent = 100.0 * wins / N_test
    print(f"\nРезультат: модель выиграла в {round(win_percent, 2)}% случаев.")

In [ ]:
game_2()

Episode 20/300 | Mean reward: 0.8199999928474426 | Loss: 0.46799999475479126
Episode 40/300 | Mean reward: 0.8399999737739563 | Loss: 0.21899999678134918
Episode 60/300 | Mean reward: 0.8999999761581421 | Loss: 0.21199999749660492
Episode 80/300 | Mean reward: 0.9599999785423279 | Loss: 0.13699999451637268
Episode 100/300 | Mean reward: 0.9800000190734863 | Loss: 0.125
Episode 120/300 | Mean reward: 0.9399999976158142 | Loss: 0.11500000208616257
Episode 140/300 | Mean reward: 0.9800000190734863 | Loss: 0.03799999877810478
Episode 160/300 | Mean reward: 0.9599999785423279 | Loss: 0.03099999949336052
Episode 180/300 | Mean reward: 0.9800000190734863 | Loss: 0.10700000077486038
Episode 200/300 | Mean reward: 0.9800000190734863 | Loss: 0.10599999874830246
Episode 220/300 | Mean reward: 1.0 | Loss: 0.10700000077486038
Episode 240/300 | Mean reward: 1.0 | Loss: 0.017000000923871994
Episode 260/300 | Mean reward: 1.0 | Loss: 0.01600000075995922
Episode 280/300 | Mean reward: 1.0 | Loss: 0.014

УРААААА ГОООООЛ, он что-то выиграл.
Это здорово, что он придумал невероятную тактику всегда выбирать героя А, но что, если правила С будет бить А, а не наоборот. Да и вообще, пора матрицу игр писать, ничью добавить...
А ещё устроим дополнительную проверку модели на адекватность - нужно будет выбирать свой ход исходя из хода соперника ("если у соперника ножницы, то очевидно нужен камень", и т.д.)

p.s. можно заметить, что это уже немного похоже на задачу, которую нам предстоит решить

ОБУЧЕНИЕ

In [ ]:
def game_3():
    ### ЭТАП 1 : МОДЕЛЬ ###

    '''
     Матрица выигрышей:
    0.5 – ничья, 1 – победа, 0 – поражение
    '''
    matrix = [[0.5, 1, 0], [0, 0.5, 1], [1, 0, 0.5]]
    def reward(a, b):
        return matrix[a][b]

    model = keras.Sequential([
        keras.layers.Input(shape=(1,)), # Вход : 1 герой соперника
        keras.layers.Embedding(input_dim=3, output_dim=4), # ДОПИСАТЬ
        keras.layers.Flatten(), # ДОПИСАТЬ
        keras.layers.Dense(3, activation='softmax')  # Выход: 3 героя, p их выбора
    ])
    optimizer = keras.optimizers.Adam(learning_rate=0.01)

    ### ЭТАП 2 : ОБУЧЕНИЕ ###
    num_episodes = 300   # Эпохи обучения
    games_per_episode = 50    # Матчей за одну эпоху

    for episode in range(num_episodes):
        with tf.GradientTape() as tape:
            # Ходы соперника
            opp_hero = np.random.randint(0, 3, size=games_per_episode)

            # Ходы модели
            state_input = tf.constant(opp_hero, dtype=tf.float32)
            state_input = tf.reshape(state_input, (games_per_episode, 1))
            probs = model(state_input, training=True)
            log_probs = tf.math.log(probs + eps)
            sampled_actions = tf.random.categorical(log_probs, num_samples=1)
            sampled_actions = tf.squeeze(sampled_actions, axis=1)

            # Награда
            rewards = []
            for i in range(games_per_episode):
                model_hero = sampled_actions[i].numpy()
                rewards.append(reward(model_hero, opp_hero[i]))

            rewards = tf.constant(rewards, dtype=tf.float32)
            chosen_action_log_p = tf.reduce_sum(
                tf.one_hot(sampled_actions, depth=3) * log_probs,
                axis=1
            )
            loss = -tf.reduce_mean(rewards * chosen_action_log_p)

        # Корректировка
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Статистика
        if (episode + 1) % 20 == 0:
            mean_reward = tf.reduce_mean(rewards)
            print(f"Episode {episode+1}/{num_episodes} | Mean reward: {round(mean_reward.numpy(), 2)} | Loss: {round(loss.numpy(), 3)}")

    ### ЭТАП 3 : ТЕСТИРОВАНИЕ ###
    print('-' * 52 + '\nТестирование\n' + '-' * 52)
    N_test = 1000
    opp_hero_test = np.random.randint(0, 3, size=N_test)
    test_input = tf.constant(opp_hero_test, dtype=tf.float32)
    test_input = tf.reshape(test_input, (N_test, 1))

    test_probs = model(test_input, training=False)
    test_log_probs = tf.math.log(test_probs + eps)
    test_actions = tf.random.categorical(test_log_probs, num_samples=1)
    test_actions = tf.squeeze(test_actions, axis=1).numpy()

    wins = 0
    L = ['A', 'B', 'C']
    for i in range(N_test):
        model_hero = test_actions[i]
        opp_hero = opp_hero_test[i]
        r = reward(model_hero, opp_hero)
        wins += r
        if i % 50 == 0 or r < 1:
            if r == 1.0:
                print(f"Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}")
            elif r:
                print(f"Модель с героем {L[model_hero]} сыграла вничью с героем соперника {L[opp_hero]}")
            else:
                print(f"Ошибка! Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}")

    win_percent = 100.0 * wins / N_test
    print(f"\nИтоговая проверка: модель выиграла в {round(win_percent, 2)}% матчей.")

In [ ]:
game_3()

Episode 20/300 | Mean reward: 0.49000000953674316 | Loss: 0.3959999978542328
Episode 40/300 | Mean reward: 0.8299999833106995 | Loss: 0.31299999356269836
Episode 60/300 | Mean reward: 0.8999999761581421 | Loss: 0.164000004529953
Episode 80/300 | Mean reward: 1.0 | Loss: 0.020999999716877937
Episode 100/300 | Mean reward: 0.9900000095367432 | Loss: 0.05900000035762787
Episode 120/300 | Mean reward: 1.0 | Loss: 0.006000000052154064
Episode 140/300 | Mean reward: 1.0 | Loss: 0.004999999888241291
Episode 160/300 | Mean reward: 1.0 | Loss: 0.004000000189989805
Episode 180/300 | Mean reward: 0.9900000095367432 | Loss: 0.06300000101327896
Episode 200/300 | Mean reward: 1.0 | Loss: 0.003000000026077032
Episode 220/300 | Mean reward: 1.0 | Loss: 0.0020000000949949026
Episode 240/300 | Mean reward: 1.0 | Loss: 0.0020000000949949026
Episode 260/300 | Mean reward: 1.0 | Loss: 0.0010000000474974513
Episode 280/300 | Mean reward: 1.0 | Loss: 0.0010000000474974513
Episode 300/300 | Mean reward: 1.0 |

Потрясающе, какие мы молодцы.
Теперь представим, что героев у нас не 3, а там, 200. А ещё мы живём в реальном мире, где и везде и повсюду вероятности, так, что есть шанс у героя в неравной схватке с другим изредка выйти победителем.

Все эти вероятности мы учтём в нашей матрице, а после этого проверим нашу модель на работоспособность в самых что ни на есть боевых условиях

In [ ]:
def game_4(matrix, hero_number=200):
    ### ЭТАП 1 : МОДЕЛЬ ###
    def reward(a, b):
        return 1 if rnd.random() < matrix[a][b] else 0

    model = keras.Sequential([
        keras.layers.Input(shape=(1,)), # Вход : 1 герой соперника
        keras.layers.Embedding(input_dim=hero_number, output_dim=hero_number+1),
        keras.layers.Flatten(),
        keras.layers.Dense(hero_number, activation='softmax')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=0.01)

    ### ЭТАП 2 : ОБУЧЕНИЕ ###
    num_episodes = 300   # Эпохи обучения
    games_per_episode = 50    # Матчей за одну эпоху

    for episode in range(num_episodes):
        with tf.GradientTape() as tape:
            # Ходы соперника
            opp_hero = np.random.randint(0, hero_number, size=games_per_episode)

            # Ходы модели
            state_input = tf.constant(opp_hero, dtype=tf.float32)
            state_input = tf.reshape(state_input, (games_per_episode, 1))
            probs = model(state_input, training=True)
            log_probs = tf.math.log(probs + eps)
            sampled_actions = tf.random.categorical(log_probs, num_samples=1)
            sampled_actions = tf.squeeze(sampled_actions, axis=1)

            # Награда
            rewards = []
            for i in range(games_per_episode):
                model_hero = sampled_actions[i].numpy()
                rewards.append(reward(model_hero, opp_hero[i]))

            rewards = tf.constant(rewards, dtype=tf.float32)
            chosen_action_log_p = tf.reduce_sum(
                tf.one_hot(sampled_actions, depth=hero_number) * log_probs,
                axis=1
            )
            loss = -tf.reduce_mean(rewards * chosen_action_log_p)

        # Корректировка
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Статистика
        if (episode + 1) % 20 == 0:
            mean_reward = tf.reduce_mean(rewards)
            print(f"Episode {episode+1}/{num_episodes} | Mean reward: {round(mean_reward.numpy(),2)} | Loss: {round(loss.numpy(),3)}")

    ### ЭТАП 3 : ТЕСТИРОВАНИЕ ###
    print('-' * 52 + '\nТестирование\n' + '-' * 52)
    N_test = 1000
    print(hero_number)
    opp_hero_test = np.random.randint(0, hero_number, size=N_test)
    test_input = tf.constant(opp_hero_test, dtype=tf.float32)
    test_input = tf.reshape(test_input, (N_test, 1))

    test_probs = model(test_input, training=False)
    test_log_probs = tf.math.log(test_probs + eps)
    test_actions = tf.random.categorical(test_log_probs, num_samples=1)
    test_actions = tf.squeeze(test_actions, axis=1).numpy()

    wins = 0
    L = [chr(i + 100) for i in range(hero_number)]
    for i in range(N_test):
        model_hero = test_actions[i]
        opp_hero = opp_hero_test[i]
        r = reward(model_hero, opp_hero)
        wins += r
        if i % 50 == 0:
            if r:
                print(f"Модель с героем {L[model_hero]} обыграла героя соперника {L[opp_hero]}")
            else:
                print(f"Ошибка! Модель с героем {L[model_hero]} проиграла герою соперника {L[opp_hero]}")

    win_percent = 100.0 * wins / N_test
    print(f"\nИтоговая проверка: модель выиграла в {round(win_percent,2)}% матчей.")

In [ ]:
hero_number = 200
matrix = [[rnd.random() * 0.8 for _ in range(hero_number)] for __ in range(hero_number)]
for i in range(hero_number):
    matrix[i][i] = 0.5
    for j in range(i+1, hero_number):
        matrix[i][j] = 1 - matrix[j][i]
game_4(matrix, hero_number)

Episode 20/300 | Mean reward: 0.6200000047683716 | Loss: 3.2869999408721924
Episode 40/300 | Mean reward: 0.47999998927116394 | Loss: 2.5269999504089355
Episode 60/300 | Mean reward: 0.46000000834465027 | Loss: 2.5179998874664307
Episode 80/300 | Mean reward: 0.5400000214576721 | Loss: 2.8589999675750732
Episode 100/300 | Mean reward: 0.6000000238418579 | Loss: 3.11299991607666
Episode 120/300 | Mean reward: 0.5199999809265137 | Loss: 2.4019999504089355
Episode 140/300 | Mean reward: 0.5600000023841858 | Loss: 2.4560000896453857
Episode 160/300 | Mean reward: 0.6000000238418579 | Loss: 2.2049999237060547
Episode 180/300 | Mean reward: 0.5400000214576721 | Loss: 1.5839999914169312
Episode 200/300 | Mean reward: 0.6000000238418579 | Loss: 1.3680000305175781
Episode 220/300 | Mean reward: 0.6399999856948853 | Loss: 0.6629999876022339
Episode 240/300 | Mean reward: 0.6399999856948853 | Loss: 0.7419999837875366
Episode 260/300 | Mean reward: 0.6600000262260437 | Loss: 0.10999999940395355
Ep

Промежуточные итоги : умеем угадывать исход матчей и выбирать оптимального героя под выбор соперника.

Дальше - больше!

# Часть 1 : Парсим датасет

In [ ]:
Heroes = []
Heroes_id = {}

with zipfile.ZipFile('/content/drive/MyDrive/data/heroes.zip', 'r') as archive:
    with archive.open('heroes.json') as f:
        heroes_data = json.load(f)
hero_num = 146
Heroes = [None] * hero_num

for hero in heroes_data:
    hero_id = hero['hero_id']
    name = hero['name']
    Heroes[hero_id] = name
    Heroes_id[name] = hero_id


print("Heroes:", Heroes)
print("Heroes_id:", Heroes_id)

Heroes: [None, 'antimage', 'axe', 'crystal-maiden', 'dazzle', 'drow-ranger', 'earthshaker', 'lich', 'lina', 'lion', 'mirana', 'morphling', 'necrophos', 'puck', 'pudge', 'razor', 'sand-king', 'shadow-shaman', 'storm-spirit', 'sven', 'tidehunter', 'vengefulspirit', 'windranger', 'witch-doctor', 'zeus', 'slardar', 'enigma', 'faceless-void', 'tiny', 'viper', 'venomancer', 'clockwerk', 'natures-prophet', 'dark-seer', 'sniper', 'beastmaster', 'pugna', 'enchantress', 'leshrac', 'shadow-fiend', 'tinker', 'weaver', 'night-stalker', 'ancient-apparition', 'spectre', 'doom', 'juggernaut', 'bloodseeker', 'riki', 'kunkka', 'chen', 'queenofpain', 'wraith-king', 'broodmother', 'huskar', 'jakiro', 'batrider', 'omniknight', 'dragon-knight', 'warlock', 'alchemist', 'life-stealer', 'death-prophet', 'ursa', 'bounty-hunter', 'silencer', 'spirit-breaker', 'invoker', 'clinkz', 'outworld-destroyer', 'bane', 'shadow-demon', 'lycan', 'lone-druid', 'brewmaster', 'phantom-lancer', 'treant', 'ogre-magi', 'gyrocopte

In [ ]:
with zipfile.ZipFile('/content/drive/MyDrive/data/parsed_matches.zip', 'r') as archive:
    with archive.open('parsed_matches.json') as f:
        matches_data = json.load(f)
print(len(matches_data), len(matches_data[0]))
print(matches_data[0])
#print(json.dumps(matches_data[0], indent=2, ensure_ascii=False))

6895 10
{'match_id': '8229805524', 'radiant_team_name': 'Силы Света', 'dire_team_name': 'Силы Тьмы', 'radiant_heroes': ['32', '7', '100', '9', '27'], 'dire_heroes': ['2', '93', '37', '95', '14'], 'radiant_players_info': [{'hero_id': 32, 'kills': 8, 'deaths': 3, 'assists': 6, 'gold_per_min': 1094, 'xp_per_min': 1516, 'last_hits': 74, 'hero_damage': 15718, 'tower_damage': 0, 'kda': ''}, {'hero_id': 7, 'kills': 1, 'deaths': 2, 'assists': 8, 'gold_per_min': 775, 'xp_per_min': 1170, 'last_hits': 78, 'hero_damage': 11488, 'tower_damage': 54, 'kda': ''}, {'hero_id': 100, 'kills': 4, 'deaths': 8, 'assists': 8, 'gold_per_min': 828, 'xp_per_min': 1323, 'last_hits': 57, 'hero_damage': 13989, 'tower_damage': 0, 'kda': ''}, {'hero_id': 9, 'kills': 2, 'deaths': 1, 'assists': 10, 'gold_per_min': 879, 'xp_per_min': 1156, 'last_hits': 92, 'hero_damage': 5310, 'tower_damage': 0, 'kda': ''}, {'hero_id': 27, 'kills': 5, 'deaths': 5, 'assists': 3, 'gold_per_min': 978, 'xp_per_min': 1152, 'last_hits': 112, 

In [ ]:
valid_counter = 0
def validation_check():
    global valid_counter, matches_data
    valid_matches = []
    for match in matches_data:
        try:
            radiant = [int(x) for x in match['radiant_heroes']]
            dire = [int(x) for x in match['dire_heroes']]
        except ValueError:
            continue
        if min(radiant + dire) == 0:
            continue
        if len(match['radiant_heroes']) != 5 or len(match['dire_heroes']) != 5:
            continue
        valid_counter += 1
        valid_matches.append(match)
    matches_data = valid_matches


def build_matrix(data):
    for match in data:
        try:
            radiant = [int(x) for x in match['radiant_heroes']]
            dire = [int(x) for x in match['dire_heroes']]
        except Exception:
            print(f'Exception in build matrix = {Exception}')
            continue
        for w in radiant:
            for l in dire:
                matrix[w][l] += 1


def normalize_matrix():
    n = len(matrix)
    for i in range(n):
        if matrix[i][i]:
            print("КАК?", i, matrix[i][i])
        for j in range(i + 1, n):
            total = matrix[i][j] + matrix[j][i]
            if total:
                matrix[i][j] = matrix[i][j] / total
                matrix[j][i] = matrix[j][i] / total


def matrix_printer(n):
    header = " " * 17
    for j in range(1, n + 1):
        header += f"Hero {j:<5}"
    print(header)
    for i in range(1, n + 1):
        row = f"Hero {i:<8}"
        for j in range(1, n + 1):
            row += f"{matrix[i][j]:10.4f}"
        print(row)

In [ ]:
validation_check()
print(f"\nКоличество валидных результатов = {valid_counter}")


Количество валидных результатов = 6836


# Часть 2 : Обучаем модель

Чем мы будем здесь заниматься?

Ну, для начала, мы хотим, чтобы модель правильно угадывала исход матчей, основываясь на персонажах в пике.

In [ ]:
rnd.shuffle(matches_data)
split_idx = int(0.8 * len(matches_data))
match_train = matches_data[:split_idx]
match_test = matches_data[split_idx:]
print(f'{len(match_train)} тренировочных и {len(match_test)} тестовых матчей')

5468 тренировочных и 1368 тестовых матчей


Постараемся воспользоваться нашими предыдущими наработками. Напишем матрицу "матчапов" героев, и постараемся обучить на ней модель.

 ДОПИСАТЬ

In [ ]:
matrix = [[0 for _ in range(hero_num)] for _ in range(hero_num)]

build_matrix(matches_data[:split_idx])
normalize_matrix()
matrix_printer(7)

                 Hero 1    Hero 2    Hero 3    Hero 4    Hero 5    Hero 6    Hero 7    
Hero 1           0.0000    0.5273    0.1818    0.2500    0.4906    0.3478    0.4098
Hero 2           0.4727    0.0000    0.4286    0.5000    0.4583    0.5128    0.4000
Hero 3           0.8182    0.5714    0.0000    0.0000    0.6667    0.4000    0.5000
Hero 4           0.7500    0.5000    1.0000    0.0000    0.4615    0.5625    0.3889
Hero 5           0.5094    0.5417    0.3333    0.5385    0.0000    0.4237    0.5102
Hero 6           0.6522    0.4872    0.6000    0.4375    0.5763    0.0000    0.4821
Hero 7           0.5902    0.6000    0.5000    0.6111    0.4898    0.5179    0.0000


P.S. Напишем ещё общую функцию, которая будет получать на вход героев и возвращать вектор из 0, 1 и -1 в зависимости от команды

In [ ]:
def vectorize_team(radiant_heroes, dire_heroes, hero_number=200):
    vector = np.zeros(hero_number)
    for hero in radiant_heroes:
        vector[int(hero) - 1] = 1
    for hero in dire_heroes:
        vector[int(hero) - 1] = -1
    return vector

Теперь к коду ДОПИСАТЬ КОММЕНТАРИИ

In [ ]:
def predict_game_1(matrix, hero_number=200):
    ### ПОДГОТОВКА ВХОДНЫХ ДАННЫХ ###
    local_hero_ids = []
    for i in range(len(matrix)):
        if any(val != 0 for val in matrix[i]):
            local_hero_ids.append(i)
    X = []
    Y = []
    num_samples = 2000
    for _ in range(num_samples):
        team1 = rnd.sample(local_hero_ids, 5)
        team2 = rnd.sample([h for h in local_hero_ids if h not in team1], 5)
        radiant_ids = [h + 1 for h in team1]
        dire_ids = [h + 1 for h in team2]
        vec = vectorize_team(radiant_ids, dire_ids, hero_number)
        X.append(vec)

        total = sum(matrix[a][b] for a in team1 for b in team2)
        Y.append(0 if total > 0 else 1)
    X = np.array(X)
    Y = np.array(Y)
    ### МОДЕЛЬ И ОБУЧЕНИЕ ###
    model = models.Sequential([
        layers.InputLayer(shape=(hero_number,)),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X, Y, epochs=60, batch_size=10, verbose=0)
    return model


def predict_test_1(model, match_test, hero_number=200):
    correct = 0
    total = 0
    for match in match_test:
        radiant = list(map(int, match['radiant_heroes']))
        dire = list(map(int, match['dire_heroes']))
        winner = 1 if match['winner'] else 0
        vector = vectorize_team(radiant, dire, hero_number)
        prediction = model.predict(vector.reshape(1, -1), verbose=0)[0][0]
        predicted_label = int(prediction >= 0.5)
        if predicted_label == winner:
            correct += 1
        total += 1
    print(f"Точность: {round((correct / total) * 100, 2)}%")

In [ ]:
model_pr_g1 = predict_game_1(matrix, hero_num)
predict_test_1(model_pr_g1, match_test, hero_num)

Точность: 55.56%
